### BertTopic


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import openai
from bertopic.representation import (
    KeyBERTInspired,
    MaximalMarginalRelevance,
    OpenAI,
    PartOfSpeech,
)

# Load your CSV data
evaluation_folder = "./curated_clotho_captions"
evaluation_csv = os.path.join(
    evaluation_folder, "clotho_captions_evaluation.csv"
)

develpment_folder = "./curated_clotho_captions"
development_csv = os.path.join(
    develpment_folder, "clotho_captions_development.csv"
)

validation_folder = "./curated_clotho_captions"
validation_csv = os.path.join(
    validation_folder, "clotho_captions_validation.csv"
)


data = pd.read_csv(development_csv)


# Extract captions columns
caption_columns = [f"caption_{i}" for i in range(1, 6)]

/home/yuvidh/micromamba/envs/audio_dl/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
def concatenate_captions(df, caption_columns):
    # Use pd.melt to transform the wide caption columns into a single concatenated column
    melted_df = pd.melt(
        df,
        id_vars=["file_name"],
        value_vars=caption_columns,
        var_name="caption_type",
        value_name="caption_text",
    )

    # Drop rows with missing captions if any
    melted_df = melted_df.dropna(subset=["caption_text"]).reset_index(
        drop=True
    )

    return melted_df[["file_name", "caption_text"]]


# Example usage:
caption_columns = [
    "caption_1",
    "caption_2",
    "caption_3",
    "caption_4",
    "caption_5",
]

# Assuming 'data' is the DataFrame you're working with
concatenated_df = concatenate_captions(data, caption_columns)

# Display the resulting DataFrame
concatenated_df["caption_text"]

0              A muddled noise of broken channel of the TV
1                 A person is turning a map over and over.
2        Several barnyard animals mooing in a barn whil...
3        An office chair is squeaking as someone bends ...
4        A flying bee is buzzing loudly around an objec...
                               ...                        
19190             Metal chimes being chimed one at a time.
19191          Going through all of the trash can noisily.
19192           A match is lit, then another match is lit.
19193       Sticks crunch and break while being walked on.
19194           Crushing up food to make a dish for others
Name: caption_text, Length: 19195, dtype: object

In [4]:
prompt = """ I have following topic labels and their keywords:
# 1. **Human Activity**: talking, people, speaking, walking, running, chatting, conversing, footsteps, crowd, conversations

# 2. **Machinery**: machine, machinery, whirring, factory, grinding, mechanical, motor, industrial

# 3. **Bird and Insect Sounds**: chirping, chirps, chirp, birds, squawking, crickets, bird, crows, quacking, singing, buzzing, wind, crickets, chimes, whistling

# 4. **Water & Water Flowing Sounds**: shower, waterfall, faucet, water, splashing, splashes, sink, flowing, fountain, dripping, pouring,drips

# 5. **Vehicle Engines**: revving, car, cars, engine, engines, vehicle, vehicles, driving, motor, drive

# 6. **Rain and Storm**: downpour, raining, rain, rainstorm, rainfall, thunderstorm, raindrops, torrential, thunder, hail, storm

# 7. **Crowd Noise with Footsteps**: crowd, crowded, talk, talks, conversations, chatter, speaks, walking, walks, walk, walked, footsteps, stepping, hiking, shoes, boots, leaves, snow, underfoot

# 9. **Door Sounds**: door, doors, hinges, opened, opens, creaking, opening, creaks, open, hinge, banging

# 10. **Train and Railway Sounds**: train, trains, locomotive, railway, railroad, rail, tracks, whistle, track, squealing, sounds, station, passing, clacking, trolley, honking

# 11. **Wind and Ocean Sounds**: windy, wind, waves, blowing, ocean, breeze, beach, shore, blows, storm, sea, gusts, whistling, gust

# 12. **General Noises and Loud Sounds**: noises, noise, sounds, sound, loud, louder, tapping, metallic, blaring, sound, while, they


Now I have a topic which contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: [KEYWORDS]

From the above 12 topic labels, assign a topic label that suits perfectly to the topic of the documents and it's keywords. Make sure it exactly is  of the format & Don't put any extra comments describing your reasoning:

topic: <topic label>
"""

In [5]:
# Load SentenceTransformer model for embeddings
embedding_model = SentenceTransformer(
    "dunzhang/stella_en_400M_v5", trust_remote_code=True
)

# UMAP and HDBSCAN models as defined
umap_model = UMAP(
    n_neighbors=18,
    n_components=6,
    min_dist=0.0,
    metric="cosine",
    random_state=42,
)
hdbscan_model = HDBSCAN(
    min_cluster_size=120,
    min_samples=90,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)

# CountVectorizer model as defined
vectorizer_model = CountVectorizer(
    stop_words="english", min_df=2, ngram_range=(1, 2)
)

# OpenAI model definition (Make sure your OpenAI setup is correct)
openai_client = openai.OpenAI(
    base_url="http://172.18.176.1:11434/v1",
    api_key="ollama",  # required, but unused
)
# prompt = """
# I have a topic that contains the following documents:
# [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a highly descriptive topic label that very well defines the topic. Make sure it is in the following format:
# topic: <topic label>, and it should not have any Topic word in it.
# """
openai_model = OpenAI(
    client=openai_client,
    model="llama3.1",
    exponential_backoff=True,
    chat=True,
    prompt=prompt,
    diversity=1,
    nr_docs=9,
)

# Representation models
representation_models = {
    "KeyBERT": KeyBERTInspired(),
    "OpenAI": openai_model,
    "MMR": MaximalMarginalRelevance(diversity=0.6),
    "POS": PartOfSpeech("en_core_web_sm"),
}

# Initialize BERTopic model without fitting yet
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_models,
    top_n_words=10,
    verbose=True,
    min_topic_size=15,
    nr_topics=13,
)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from collections import Counter

caption_column = "all_caps"
# Extract the captions from the current column
captions = concatenated_df["caption_text"].tolist()

# Generate embeddings for captions
embeddings = embedding_model.encode(captions, show_progress_bar=True)

# Fit the BERTopic model to the current column's captions
topics, probs = topic_model.fit_transform(captions, embeddings)
# topic_model.reduce_topics(captions, nr_topics=10)
# Get the topic labels
topic_labels = topic_model.get_topic_info()
topic_labels_dict = topic_labels.set_index("Topic")["Name"].to_dict()

# Handle -1 topics (documents not assigned to any topic)
topic_labels_dict[-1] = "IRRelevant"

# Add topic labels and representations to the original DataFrame
concatenated_df[f"{caption_column}_topic"] = [
    topic_labels_dict.get(topic, "No topic assigned") for topic in topics
]
concatenated_df[f"{caption_column}_KeyBERT"] = [
    (
        topic_labels.loc[topic_labels["Topic"] == topic, "KeyBERT"].values[0]
        if topic in topic_labels_dict
        else "No topic assigned"
    )
    for topic in topics
]
concatenated_df[f"{caption_column}_OpenAI"] = [
    (
        topic_labels.loc[topic_labels["Topic"] == topic, "OpenAI"].values[0]
        if topic in topic_labels_dict
        else "No topic assigned"
    )
    for topic in topics
]
concatenated_df[f"{caption_column}_MMR"] = [
    (
        topic_labels.loc[topic_labels["Topic"] == topic, "MMR"].values[0]
        if topic in topic_labels_dict
        else "No topic assigned"
    )
    for topic in topics
]
concatenated_df[f"{caption_column}_POS"] = [
    (
        topic_labels.loc[topic_labels["Topic"] == topic, "POS"].values[0]
        if topic in topic_labels_dict
        else "No topic assigned"
    )
    for topic in topics
]

# Count the number of occurrences of each topic
topic_counts = Counter(topics)

# Store the topic counts for the current caption column
all_topic_counts = {
    topic: {
        "Count": count,
        "KeyBERT": (
            topic_labels.loc[topic_labels["Topic"] == topic, "KeyBERT"].values[
                0
            ]
            if topic in topic_labels_dict
            else "No topic assigned"
        ),
        "OpenAI": (
            topic_labels.loc[topic_labels["Topic"] == topic, "OpenAI"].values[
                0
            ]
            if topic in topic_labels_dict
            else "No topic assigned"
        ),
        "MMR": (
            topic_labels.loc[topic_labels["Topic"] == topic, "MMR"].values[0]
            if topic in topic_labels_dict
            else "No topic assigned"
        ),
        "POS": (
            topic_labels.loc[topic_labels["Topic"] == topic, "POS"].values[0]
            if topic in topic_labels_dict
            else "No topic assigned"
        ),
    }
    for topic, count in topic_counts.items()
}

Batches: 100%|██████████| 600/600 [00:19<00:00, 30.76it/s]
2024-09-09 13:44:40,240 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
/home/yuvidh/micromamba/envs/audio_dl/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
2024-09-09 13:44:57,750 - BERTopic - Dimensionality - Completed ✓
2024-09-09 13:44:57,751 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process ju

In [7]:
# Convert the dictionary into a DataFrame
def create_topic_df(topic_dict):
    # Initialize an empty list to hold the rows
    rows = []

    # Iterate through the dictionary to create rows for the DataFrame
    for topic_num, topic_info in topic_dict.items():
        # Create a row for each topic
        row = {
            "Topic": topic_num,
            "Count": topic_info["Count"],
            "KeyBERT": ", ".join(topic_info["KeyBERT"]),
            "OpenAI": ", ".join(topic_info["OpenAI"]),
            "MMR": ", ".join(topic_info["MMR"]),
            "POS": ", ".join(topic_info["POS"]),
        }
        rows.append(row)

    # Create the DataFrame
    df = pd.DataFrame(rows)
    return df


# Create the DataFrame
topic_df = create_topic_df(all_topic_counts)

In [8]:
topic_df.to_csv("topics_reduced.csv")

In [30]:
import pandas as pd

# Create a DataFrame from the all_topic_counts dictionary
topic_counts_df = pd.DataFrame(
    {
        column: pd.Series(topic_counts)
        for column, topic_counts in all_topic_counts.items()
    }
)

# Reset the index to create a multi-index DataFrame
topic_counts_df = topic_counts_df.stack().reset_index()
topic_counts_df.columns = ["Topic", "Caption Column", "Topic Info"]

# # Convert the 'Topic Info' column to a dictionary
# topic_counts_df['Topic Info'] = topic_counts_df['Topic Info'].apply(lambda x: dict(x))

# # Expand the 'Topic Info' dictionary into separate columns
# topic_counts_df = pd.concat([topic_counts_df.drop('Topic Info', axis=1), topic_counts_df['Topic Info'].apply(pd.Series)], axis=1)

# # Rename the columns
# topic_counts_df.columns = ['Topic', 'Caption Column', 'Count', 'KeyBERT', 'OpenAI', 'MMR', 'POS']

In [ ]:
topic_counts_df

In [8]:
df = topic_counts_df.copy()

In [10]:
df.to_csv("./curated_clotho_captions/topics_development.csv")

In [ ]:
import pandas as pd

# Assuming 'df' is your existing DataFrame
# Split the DataFrame into separate DataFrames for each unique 'Caption Column'
unique_captions = df["Caption Column"].unique()

# Create a dictionary to store the separate DataFrames
caption_dfs = {}

for caption in unique_captions:
    caption_dfs[caption] = df[df["Caption Column"] == caption].reset_index(
        drop=True
    )

# Now, caption_dfs will contain separate DataFrames for each 'Caption Column'
# For example, you can access the DataFrame for 'caption_1' using:
df_caption_1 = caption_dfs["caption_1"]

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag

# Download necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")


def create_combined_wordcloud(df, ignore_topic="-1", representation="KeyBERT"):
    # Validate ignore_topic and representation
    if ignore_topic not in df["Topic"].unique():
        raise ValueError(
            f"Invalid ignore_topic '{ignore_topic}'. It should be one of {df['Topic'].unique()}"
        )
    if representation not in ["KeyBERT", "OpenAI", "MMR", "POS"]:
        raise ValueError(
            f"Invalid representation '{representation}'. It should be one of ['KeyBERT', 'OpenAI', 'MMR', 'POS']"
        )

    stop_words = set(stopwords.words("english"))

    col_mapping = {
        "KeyBERT": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
        "OpenAI": lambda x: [word for word in x],
        "MMR": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
        "POS": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
    }

    # Filter the DataFrame
    df_filtered = df[df["Topic"] != ignore_topic]

    # Apply the mapping function to the specified column
    all_words = []
    for _, row in df_filtered.iterrows():
        words = col_mapping[representation](row[representation])
        all_words.extend(
            words * row["Count"]
        )  # Use Count as weight for the words

    # Calculate word frequencies
    word_freqs = Counter(all_words)

    # Generate and display the word cloud
    plt.figure(figsize=(12, 8))
    wc = WordCloud(
        width=800,
        height=500,
        max_font_size=110,
        background_color="white",
        colormap="viridis",
    ).generate_from_frequencies(word_freqs)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"{representation} Representation")
    plt.show()

In [ ]:
create_combined_wordcloud(
    caption_dfs["caption_1"], ignore_topic=-1, representation="OpenAI"
)
create_combined_wordcloud(
    caption_dfs["caption_2"], ignore_topic=-1, representation="OpenAI"
)
create_combined_wordcloud(
    caption_dfs["caption_3"], ignore_topic=-1, representation="OpenAI"
)
create_combined_wordcloud(
    caption_dfs["caption_4"], ignore_topic=-1, representation="OpenAI"
)
create_combined_wordcloud(
    caption_dfs["caption_5"], ignore_topic=-1, representation="OpenAI"
)

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
import matplotlib as mpl

# Download necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")


def create_combined_wordcloud(df, ignore_topic="-1", representation="KeyBERT"):
    # Validate ignore_topic and representation
    if ignore_topic not in df["Topic"].unique():
        raise ValueError(
            f"Invalid ignore_topic '{ignore_topic}'. It should be one of {df['Topic'].unique()}"
        )
    if representation not in ["KeyBERT", "OpenAI", "MMR", "POS"]:
        raise ValueError(
            f"Invalid representation '{representation}'. It should be one of ['KeyBERT', 'OpenAI', 'MMR', 'POS']"
        )

    stop_words = set(stopwords.words("english"))

    col_mapping = {
        "KeyBERT": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
        "OpenAI": lambda x: [word for word in x],
        "MMR": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
        "POS": lambda x: [
            word
            for sentence in x
            for word, _ in pos_tag(word_tokenize(sentence))
            if word.lower() not in stop_words
        ],
    }

    # Filter the DataFrame
    df_filtered = df[df["Topic"] != ignore_topic]

    # Apply the mapping function to the specified column
    all_words = []
    for _, row in df_filtered.iterrows():
        words = col_mapping[representation](row[representation])
        all_words.extend(
            words * row["Count"]
        )  # Use Count as weight for the words

    # Calculate word frequencies
    word_freqs = Counter(all_words)

    # Generate the word cloud
    wc = WordCloud(
        width=800,
        height=500,
        max_font_size=110,
        background_color="white",
        colormap="viridis",
    ).generate_from_frequencies(word_freqs)

    # Create figure and axis
    fig, ax = plt.subplots(1, 2, figsize=(14, 8))

    # Plot the word cloud
    ax[0].imshow(wc, interpolation="bilinear")
    ax[0].axis("off")
    ax[0].set_title(f"{representation} Representation")

    # Create color bar
    color_map = plt.get_cmap("viridis")
    norm = mpl.colors.Normalize(
        vmin=min(word_freqs.values()), vmax=max(word_freqs.values())
    )
    sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
    sm.set_array([])  # Only needed for older versions of Matplotlib

    # Plot the color bar
    cbar = fig.colorbar(
        sm, ax=ax[1], orientation="vertical", fraction=0.02, pad=0.04
    )
    ax[1].set_title("Word Frequency Color Bar")

    plt.show()

In [ ]:
# ### Common Sounds
# Sure, here are the unique topics derived from the captions:

# 1. **Human Activity**: talking, people, speaking, walking, running, chatting, conversing, footsteps, crowd, conversations

# 2. **Machinery**: machine, machinery, whirring, factory, grinding, mechanical, motor, industrial

# 3. **Bird and Insect Sounds**: chirping, chirps, chirp, birds, squawking, crickets, bird, crows, quacking, singing, buzzing, wind, crickets, chimes, whistling

# 4. **Water & Water Flowing Sounds**: shower, waterfall, faucet, water, splashing, splashes, sink, flowing, fountain, dripping, pouring,drips

# 5. **Vehicle Engines**: revving, car, cars, engine, engines, vehicle, vehicles, driving, motor, drive

# 6. **Rain and Storm**: downpour, raining, rain, rainstorm, rainfall, thunderstorm, raindrops, torrential, thunder, hail, storm

# 7. **Crowd Noise with Footsteps**: crowd, crowded, talk, talks, conversations, chatter, speaks, walking, walks, walk, walked, footsteps, stepping, hiking, shoes, boots, leaves, snow, underfoot

# 9. **Door Sounds**: door, doors, hinges, opened, opens, creaking, opening, creaks, open, hinge, banging

# 10. **Train and Railway Sounds**: train, trains, locomotive, railway, railroad, rail, tracks, whistle, track, squealing, sounds, station, passing, clacking, trolley, honking

# 11. **Wind and Ocean Sounds**: windy, wind, waves, blowing, ocean, breeze, beach, shore, blows, storm, sea, gusts, whistling, gust

# 12. **General Noises and Loud Sounds**: noises, noise, sounds, sound, loud, louder, tapping, metallic, blaring, sound, while, they